<a href="https://colab.research.google.com/github/betto9711/Diplomado/blob/main/NOTAS/Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA

In [ ]:
def plot_confusion_matrix(cm, labels):
    fig_cm = px.imshow(cm, labels=dict(x="Predicted", y="Actual", color="Count"),
                       x=labels, y=labels, color_continuous_scale='Viridis', text_auto = True,
                       title="Confusion Matrix")
    fig_cm.update_layout(coloraxis_showscale=False)
    fig_cm.show()

In [ ]:
breast_cancer = load_breast_cancer()
df = pd.DataFrame(data=breast_cancer.data, columns=breast_cancer.feature_names)
df['target'] = breast_cancer.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [ ]:
X = breast_cancer.data  # Features
y = breast_cancer.target  # Labels

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_scaled)
pca_df_3d = pd.DataFrame(data=X_pca_3d, columns=['PC1', 'PC2', 'PC3'])
pca_df_3d['diagnosis'] = y
pca_df_3d.head()

,PC1,PC2,PC3,diagnosis
0,9.192837,1.948584,-1.123169,0
1,2.387802,-3.768172,-0.529294,0
2,5.733896,-1.075174,-0.551748,0
3,7.122953,10.275589,-3.232790,0
4,3.935302,-1.948071,1.389764,0


In [ ]:
fig_3d = px.scatter_3d(pca_df_3d, x='PC1', y='PC2', z='PC3', color='diagnosis', template = 'plotly_white', title = 'PCA - 3 Components')
fig_3d.show()

In [ ]:
explained_variance_3d = pca_3d.explained_variance_ratio_
print("Explained Variance Ratio (3D):", explained_variance_3d)

Explained Variance Ratio (3D): [0.44272026 0.18971182 0.09393163]


In [ ]:
components = pd.DataFrame(abs(pca_3d.components_), columns=breast_cancer.feature_names)
components

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,0.218902,0.103725,0.227537,0.220995,0.142590,0.239285,0.258400,0.260854,0.138167,0.064363,...,0.227997,0.104469,0.236640,0.224871,0.127953,0.210096,0.228768,0.250886,0.122905,0.131784
1,0.233857,0.059706,0.215181,0.231077,0.186113,0.151892,0.060165,0.034767,0.190349,0.366575,...,0.219866,0.045467,0.199878,0.219352,0.172304,0.143593,0.097964,0.008257,0.141883,0.275339
2,0.008531,0.064550,0.009314,0.028700,0.104292,0.074092,0.002734,0.025564,0.040240,0.022574,...,0.047507,0.042298,0.048546,0.011902,0.259798,0.236076,0.173057,0.170344,0.271313,0.232791


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df['target'], test_size=0.2, random_state=7)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
pca = PCA(n_components=3)
X_train_pca_2d = pca.fit_transform(X_train_scaled)
X_test_pca_2d = pca.transform(X_test_scaled)

In [ ]:
lr_pca_2d = LogisticRegression()
lr_pca_2d.fit(X_train_pca_2d, y_train)
lr_pca_2d_pred = lr_pca_2d.predict(X_test_pca_2d)

In [ ]:
lr_pca_2d_accuracy = accuracy_score(y_test, lr_pca_2d_pred)
lr_pca_2d_precision = precision_score(y_test, lr_pca_2d_pred)
lr_pca_2d_recall = recall_score(y_test, lr_pca_2d_pred)
lr_pca_2d_f1 = f1_score(y_test, lr_pca_2d_pred)
lr_pca_2d_report = classification_report(y_test, lr_pca_2d_pred)
print("Logistic Regression PCA 2D Classification Report:")
print(lr_pca_2d_report)

Logistic Regression PCA 2D Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.80      0.88        40
           1       0.90      0.99      0.94        74

    accuracy                           0.92       114
   macro avg       0.94      0.89      0.91       114
weighted avg       0.93      0.92      0.92       114

